In [2]:
# NiN网络: 网络中的网络. 
# 首先导入：包
import torch
from torch import nn
from d2l import torch as d2l


# 首先我们实现一下nin块
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        # 然后跟上两个1*1的卷积层. 不改变输出通道, K=1, p=0, S=1,这样我们不改变输入和输出通道的形状.
        nn.Conv2d(1,  1, kernel_size=1, stride=1, padding=0), nn.ReLU(), 
        nn.Conv2d(1, 1, kernel_size=1, stride=1, padding=0), nn.ReLU()


    )


In [ ]:
# 构建NiN模型: NiN是基于AlexNet架构的. 
# 就是通道数变化都差不太多的.
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0)

)